In [168]:
import numpy as np
from ipywidgets import *
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt

In [169]:
# AMBIGUITY
def range_amb(Rn, Rf, pri):
    return (2*Rf/c <= (2*Rn/c) + pri)

def azi_amb(pri, swath, theta, vs, antenna_azi):
    return (pri >= 2*swath*np.sin(theta)/c)&(pri<=antenna_azi/(2*vs))

def antenna_H(R, prf, lamda, Rn, theta):
    return (R >= (2*lamda*prf*Rn*np.tan(theta)/c))

def prf_select(j,Rn, Rf, tp, tpt, R, prf):
    return (prf >= (j/((2*Rn/c) - tp - tpt - (2*R/c))))&(prf >= ((j+1)/((2*Rf/c) - tp - tpt - (2*R/c))))


In [170]:
def ffy(s):
    return np.fft.fftshift(np.fft.fft(np.fft.fftshift(s)))
def iffy(fs):
    return np.fft.ifftshift(np.fft.ifft(np.fft.ifftshift(fs)))


In [171]:
def db2mag(x):
    return np.power(10,(x/10))
def mag2db(y):
    return 10*np.log10(y)

In [172]:
# orital parameters
h = 400e+03 # obital height (m)
G = 6.6743e-11 # Gravitational Constant (SI)
Re = 6.3781e+06 # radius of earth (m)
Me = 5.97219e+24 # mass of the earth (kgs)

In [173]:
vs = np.sqrt(G*Me/(h+Re)) # orbital speed of Satellite @ h
print("%0.3f" % (vs/1000), "km/s")

7.669 km/s


In [174]:
c = 2.99e8 # speed of light (m/s)

In [175]:
no_of_pulse = 32
fc = 9e9 # carier frequency (Hz)
duty_cycle = 0.30 # duty cycle of transmitted signal
pri = 700e-6 # pulse repetition interval(s)
prf = 1/pri # pulse repetition frequency (Hz)
bw = 200e6 # baseband bandwidth (hz)
wo = 2*np.pi*bw # radian/s baseband
# snr = 16 # recived signal to noise ratio (db)
t_start = 0 # start time of simulation
t_stop = no_of_pulse*pri # end time of simulation
k = wo/c # wavenumber (for both path)
lamda = 2*np.pi/k # wavelength
pw = duty_cycle*pri # pulse width (s)
wc = np.pi*2*fc # radian/s carrier 
tp = pw # pulse time
tpt = 0.01*pw # protection time 

In [176]:
alpha = wo/pw # chirp rate
beta = wc-alpha*pw # modified chirp carrier
print("%0.3f"% alpha, "%0.3f"% beta)

5983986006837.701 55292030703.180


In [177]:
look_angle = 18*np.pi/180 # look angle (rad)
R = (h+Re)*np.cos(look_angle) - np.sqrt(((h+Re)*np.cos(look_angle))**2 - (2*h*Re) - h**2) # target range
print("%0.3f"% (R/1000), "km")

421.987 km


In [178]:
Xc = np.sqrt((h**2) + (R**2) - (2*R*h*np.cos(look_angle))) # centre of the range
print("Xc = ","%0.3f"% (Xc/1000), "km")

Xc =  130.408 km


In [179]:
antenna_azi = 12 # antenna length in azimuth direction (m)
antenna_range = 4 # antenna length in range direction (m)

theta = look_angle + np.arcsin((R/Re)*np.sin(look_angle)) # incident angle (angle between a beam and the 
print("%0.3f"% (theta*180/np.pi), "degree") # perpendicular through earth center i.e. perpendicular to surface)


azi_res = antenna_azi/2 # azimuth resolution (m)
thetaR = lamda/antenna_range # angular range 3dB beamwidth of antenna (rad)
thetaA = lamda/antenna_azi # angular azimuth 3dB beamwidth of antenna (rad)

19.172 degree


In [180]:
delta_x = c/(4*bw) # range resolution
delta_t = np.pi/(2*alpha*pw) # Time domain sampling sepration
print(delta_t, delta_x)

1.25e-09 0.37375


In [181]:
swath = thetaR*R*np.tan(theta)
Xo = swath/2
Rn = Xc-Xo # near target
Rf = Xc+Xo # far target

Ts = 2*Rn/c # start time of recieving window
Tf = 2*Rf/c + tp # end time of recieving window
Tx = 4*Xo/c # Range swath echo time 
print("Ts = %.6f" %Ts)
print("Tf = %.6f" %Tf)
print("Tx = %.6f" %Tx)

print("swath = ", swath/1000, "km")
print("Xo = ", Xo/1000,"km")

Ts = 0.000689
Tf = 0.001266
Tx = 0.000367
swath =  54.835103594699 km
Xo =  27.4175517973495 km


In [182]:
range_res = c/(2*bw*np.sin(theta)) # improved range resolution (m)
gnd_range_res = range_res/np.sin(theta) # ground range resolution (m)
Ls = thetaA*(Xc-Xo) # synthetic aperture length (m)
ts = Ls/c # synthetic aperture time (s)
fd = 2*vs**2/(lamda*(Xc-Xo)) # doppler frequency rate (Hz)
Ba = ts*fd # azimuth bandwidth
antenna_area = antenna_azi*antenna_range # antenna area (m^2)

print(lamda)
print("range resolution = ", "%0.3f"% range_res, "m")
print("ground range resolution = ", "%0.3f"% gnd_range_res, "m")

1.495
range resolution =  2.276 m
ground range resolution =  6.931 m


In [183]:
k = 4 # k is greater than equal to 4 for the range and azimuth ambiguity satisfaction
kb = 1.38e-23 # boltzmann constant
antenna_area_min=k*vs*R*np.tan(theta)/c
print("%0.3f"% antenna_area_min, "m^2")
# print(thetaR, thetaA, swath/1000)

# assumption
Peak_Power = 600 # watt
avg_power = 2*alpha*Peak_Power
noise_bw = 1/(2*alpha*pw)
norm_bsc = -20 # normalized backscattering constant
noise_temp = 290 # in kelvin
antenna_gain = 33 # antenna gain (db)
systemNoiseFigure = 4 # system noise figure (db)
randLoss = 1 # random loss (db)
TargetProccLoss = 4 # random processing Losses
clutterProccLoss = 1 # Clutter Proccessing Losses (db)
sys_loss = systemNoiseFigure + randLoss + TargetProccLoss + clutterProccLoss
noise_factor = -20 # noise factor (db)

# Noise and SNR
noise_power = mag2db(kb) + noise_factor + noise_temp + noise_bw # noise_bw is noise equivalent bandwidth in Hz
snr = (mag2db(avg_power) + 2*antenna_gain + 3*mag2db(lamda) + norm_bsc
       + mag2db(gnd_range_res) - mag2db(2*((4*np.pi*R)**3)*vs) + mag2db(1.38e-23) + noise_factor + noise_temp + sys_loss)

nesz = -1*snr
print(noise_power, snr, nesz)
                                                                 

15.052 m^2
41.39879086441026 26.015161843966325 -26.015161843966325


In [184]:
# ambiguity checks 
for j in np.arange(1,no_of_pulse+1,1):
    if prf_select(j,Rn, Rf, pw, tpt, R, prf) == False:
        print("prf Assumption not satisfied")
        break
if j == no_of_pulse:
    print("prf assumption satisfied")
print(range_amb(Rn, Rf, pri))
print(azi_amb(pri, swath, theta, vs, antenna_azi))
print(antenna_H(R, prf, lamda, Rn, theta))

prf assumption satisfied
True
True
True
